## 构建ssg 索引

构建ssg索引的原始数据：
1. npy数据文件：二维数组，每行表示一个向量。
2. idx索引文件：kgraph索引文件；

默认的kgraph 配置：https://github.com/gfhe/kgraph/blob/master/doc/params.md#index-parameters

```
K	25	
L	100
S	10
R	100
iter    30
```



构建步骤:
1. 编译ssg
2. npy转fvecs
3. test_ssg_index 命令构建ssg索引.
    * SSG 的默认`angle=60`

In [3]:
import numpy as np

def npy_to_fvecs(npy_file, fvecs_file):
    # 从npy文件中加载数组
    array = np.load(npy_file)

    # 打开fvecs文件进行写入
    with open(fvecs_file, 'wb') as f:
        # 获取数组的形状
        num_vectors, vector_dim = array.shape

        # 将每个向量写入fvecs文件
        for vector in array:
            # 每行开始，向量的维度
            f.write(np.array(vector_dim, dtype=np.int32).tobytes())
            
            # 将向量转换为float32类型，并写入fvecs文件
            f.write(vector.astype(np.float32).tobytes())

    print("转换完成！")

# 调用函数进行转换
npy_file = "data/10000-sift1m.npy"      # 输入的npy文件路径
fvecs_file = "data/10000-sift1m.fvecs"  # 输出的fvecs文件路径   
npy_to_fvecs(npy_file, fvecs_file)

转换完成！


In [5]:
# 验证转换数据的正确性

def ivecs_read(fname):
    a = np.fromfile(fname, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy()


def fvecs_read(fname):
    return ivecs_read(fname).view('float32')


dataset = fvecs_read('data/10000-sift1m.fvecs')
dataset.shape

(10000, 128)

## 使用struct 库读区ssg 索引
参考：https://poe.com/s/oYR9KsWm9wnmjFyIRGQI